In [1]:
import os
import hw
import env_ingredient_add
def get_scene():
    if os.getcwd() == "/datasets/_deepnote_work/manipulation/project": 
        #scene = open("/work/manipulation/project/objects/environment_setup.yaml")
        scene = env_ingredient_add.get_environment_set_up(no_scene=True,include_driver=True)
        xmls = [os.getcwd() + "/package.xml", "/work/manipulation/project/package.xml"]
    else:
        #scene = open("objects/environment_setup.yaml") # local setup
        scene = env_ingredient_add.get_environment_set_up(no_scene=True,include_driver=True)
        xmls = [os.getcwd() + "/package.xml"]

    return scene

In [2]:
scenario = hw.LoadScenario(data=get_scene())
meshcat = hw.StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [14]:
# at present, trajectory blank doesn't work
diagram,sim = hw.init_diagram(meshcat,scenario)
# hw.print_diagram(diagram)

context = diagram.CreateDefaultContext()
traj = hw.create_small_trajectory(diagram,meshcat,context)

meshcat.Delete()
diagram,sim = hw.init_diagram(meshcat,scenario,traj)
controller = diagram.GetSubsystemByName("diff_ik_integrator")

# hw.fix_base_pos(controller.get_parameters(), 2*[False] + [True])


In [15]:
hw.fix_base_pos(controller.get_parameters(), 3*[True])

In [16]:


hw.create_small_trajectory(diagram,meshcat)

# hw.run_simulation(meshcat,sim,2)



# print_diagram(diagram)

hw.run_simulation(meshcat,sim,20)

In [10]:
plant = diagram.GetSubsystemByName("station").GetSubsystemByName("plant")

for index in plant.GetJointIndices():
    print(plant.get_joint(index).name())

iiwa_base_x
iiwa_base_y
iiwa_base_z
iiwa_joint_1
iiwa_joint_2
iiwa_joint_3
iiwa_joint_4
iiwa_joint_5
iiwa_joint_6
iiwa_joint_7
iiwa_joint_ee
tool0_joint
left_finger_sliding_joint
right_finger_sliding_joint
iiwa_link_7_welds_to_body


In [ ]:


print(controller.get_parameters().get_joint_position_limits())

print(f"after running: {controller.get_parameters().get_joint_position_limits()}")

hw.run_simulation(meshcat,sim,10)